# Evaluamos el clasificador

Los mejores resultados se obtuvieron usando el clasificador LinearSVC, por lo que se hará análisis de error sobre esa variante.

In [2]:
%run scripts/eval.py -c ../ancora-3.0.1es -i ../class_svm -m


Accuracy: 94.11% / 97.57% / 62.76% (total / known / unk)

g \ m	sp000	nc0s000	da0000	aq0000	fc	nc0p000	rg	np00000	fp	cc
sp000	14.30	0.00	-	0.03	-	-	0.00	-	-	-	
nc0s000	0.00	12.09	0.01	0.33	-	0.02	0.02	0.08	-	0.00	
da0000	-	0.08	9.53	-	-	0.00	-	0.00	-	-	
aq0000	0.00	0.34	-	6.61	-	0.15	0.01	0.05	-	-	
fc	-	-	-	-	5.85	-	-	-	-	-	
nc0p000	-	0.25	-	0.31	-	4.90	-	0.04	-	-	
rg	0.02	0.02	0.00	0.21	-	0.01	3.35	0.01	-	0.02	
np00000	0.00	0.21	-	0.08	-	0.01	-	3.26	-	0.00	
fp	-	-	-	-	-	-	-	-	3.55	-	
cc	0.00	-	-	0.01	-	-	0.05	0.00	-	3.34	


Los resultados parecen ser bastante buenos, fuera de la diagonal no tenemos algún valor considerablemente alto. 

# 1er cambio:

Uno de los errores más altos que tenemos (por encima del 0.2) es que el modelo confunde nombres comunes plurales como singulares. Un feature simple de agregar sería ver si una palabra termina o no en 's'. Agrego dicho feature tanto para la palabra actual, como para la siguiete y la anterior y vuelvo a evaluar.

In [4]:
%run scripts/eval.py -c ../ancora-3.0.1es -i ../class_svm_1 -m


Accuracy: 94.49% / 97.59% / 66.42% (total / known / unk)

g \ m	sp000	nc0s000	da0000	aq0000	fc	nc0p000	rg	np00000	fp	cc
sp000	14.30	0.01	-	0.02	-	0.00	0.00	-	-	-	
nc0s000	0.00	12.14	0.01	0.29	-	0.01	0.02	0.08	-	0.00	
da0000	-	0.07	9.53	0.00	-	0.00	-	0.01	-	-	
aq0000	0.00	0.35	-	6.50	-	0.27	0.01	0.05	-	-	
fc	-	-	-	-	5.85	-	-	-	-	-	
nc0p000	-	0.01	-	0.18	-	5.30	-	0.02	-	-	
rg	0.02	0.03	0.00	0.18	-	0.02	3.35	0.01	-	0.02	
np00000	0.00	0.20	-	0.06	-	0.02	-	3.28	-	0.00	
fp	-	-	-	-	-	-	-	-	3.55	-	
cc	0.00	0.00	-	0.00	-	0.01	0.05	0.00	-	3.34	


Mejoró el accuracy ~4% sobre palabras desconocidas y en ese error en particular se pasó de 0.25 a 0.01. El accuracy total y sobre palabras conocidas no se vió afectado (de hecho, subió levemente). **Debido a esto, conservo el cambio para el resto del análisis.**

Otro error es confundir nombres propios por nombres comunes.

In [5]:
sorted(error_sents['np00000']['nc0s000'])[:10]

[4, 6, 9, 12, 29, 36, 37, 47, 61, 68]

In [39]:
sent_numb = 68
sent = sents[sent_numb]
words, tags = zip(*sent)  # separar las palabras de los tags
wrong_tags = [word[0] for word in sent if word[1] == 'np00000']
wrong_tags

['Anna_Birulés', 'Investigación_y_Desarrollo', 'Gobierno']

In [37]:
model.tag(wrong_tags)

array(['np00000', 'aq0000', 'np00000'], dtype='<U8')

# 2do cambio:

Luego de los nombres que se etiquetan mal, parecen coincider en que contienen algún guión bajo. Agrego como feature ver si hay algún guión bajo en la palabra, tanto para la actual como la siguiente y anterior. Además, agrego lo mismo para para guión medio. Volvemos a evaluar por separado y en combinación.

Guión medio:

In [45]:
%run scripts/eval.py -c ../ancora-3.0.1es -i ../class_svm_2_1 -m


Accuracy: 94.49% / 97.59% / 66.41% (total / known / unk)

g \ m	sp000	nc0s000	da0000	aq0000	fc	nc0p000	rg	np00000	fp	cc
sp000	14.30	0.01	-	0.02	-	0.00	0.00	-	-	-	
nc0s000	0.00	12.14	0.01	0.30	-	0.01	0.02	0.07	-	0.00	
da0000	-	0.07	9.53	0.00	-	0.00	-	0.01	-	-	
aq0000	0.00	0.35	-	6.50	-	0.27	0.01	0.05	-	-	
fc	-	-	-	-	5.85	-	-	-	-	-	
nc0p000	-	0.01	-	0.18	-	5.30	-	0.02	-	-	
rg	0.02	0.03	0.00	0.18	-	0.02	3.35	0.00	-	0.02	
np00000	0.00	0.20	-	0.06	-	0.02	-	3.28	-	0.00	
fp	-	-	-	-	-	-	-	-	3.55	-	
cc	0.00	0.00	-	0.00	-	0.01	0.05	0.00	-	3.34	


Guión bajo:

In [44]:
%run scripts/eval.py -c ../ancora-3.0.1es -i ../class_svm_2_2 -m


Accuracy: 94.18% / 97.28% / 66.08% (total / known / unk)

g \ m	sp000	nc0s000	da0000	aq0000	fc	nc0p000	rg	np00000	fp	cc
sp000	14.22	0.04	-	0.07	-	0.00	0.00	-	-	-	
nc0s000	0.00	12.20	0.01	0.27	-	0.01	0.02	0.05	-	0.00	
da0000	-	0.12	9.48	-	-	0.01	-	0.00	-	-	
aq0000	0.00	0.39	-	6.49	-	0.27	0.01	0.03	-	-	
fc	-	-	-	-	5.85	-	-	-	-	-	
nc0p000	-	0.01	-	0.19	-	5.30	-	0.01	-	-	
rg	0.02	0.06	0.00	0.21	-	0.03	3.29	0.00	-	0.02	
np00000	-	0.26	-	0.06	-	0.05	-	3.19	-	0.00	
fp	-	-	-	-	-	-	-	-	3.55	-	
cc	-	0.01	-	0.00	-	0.01	0.05	0.00	-	3.34	


Guión medio y guión bajo:

In [43]:
%run scripts/eval.py -c ../ancora-3.0.1es -i ../class_svm_2_3 -m


Accuracy: 94.18% / 97.28% / 66.08% (total / known / unk)

g \ m	sp000	nc0s000	da0000	aq0000	fc	nc0p000	rg	np00000	fp	cc
sp000	14.22	0.04	-	0.07	-	0.00	0.00	-	-	-	
nc0s000	0.00	12.20	0.01	0.27	-	0.01	0.02	0.05	-	0.00	
da0000	-	0.12	9.48	-	-	0.01	-	0.00	-	-	
aq0000	0.00	0.39	-	6.49	-	0.27	0.01	0.03	-	-	
fc	-	-	-	-	5.85	-	-	-	-	-	
nc0p000	-	0.01	-	0.19	-	5.30	-	0.01	-	-	
rg	0.02	0.06	0.00	0.21	-	0.03	3.29	0.00	-	0.02	
np00000	-	0.26	-	0.06	-	0.05	-	3.19	-	0.00	
fp	-	-	-	-	-	-	-	-	3.55	-	
cc	-	0.01	-	0.00	-	0.01	0.05	0.00	-	3.34	


Ninguna combinación mejoró el rendimiento del modelo. De hecho, dos de ellas lo redujeron. **Debido a esto, no se conservan los cambios**